In [1]:
import pandas as pd
import os
import sentencepiece as spm
from time import time

In [2]:
data_path = "../../data/four_channels.csv"
data = pd.read_csv(data_path)

data

,id,source,date,text,views,forwards,containing_media,👍,❤,🤬,😢,😁
0,19189,Times of Ukraine,2023-07-24 13:20:42,🧡 Кадрові зміни у Кабміні. Новим керівником Мі...,84755,80,True,869.0,23.0,14.0,2.0,NaN
1,19188,Times of Ukraine,2023-07-24 12:36:37,"💚 Не минуло і місяця після звільнення з ЗСУ, я...",104968,218,True,622.0,80.0,277.0,21.0,NaN
2,19187,Times of Ukraine,2023-07-24 11:41:15,У Польщі запрацював хаб з ремонту танків Leopa...,115646,77,True,2513.0,361.0,10.0,NaN,NaN
3,19186,Times of Ukraine,2023-07-24 11:20:06,У Києві затримали колишнього віськкома-втікача...,120479,245,True,2639.0,23.0,8.0,4.0,NaN
4,19185,Times of Ukraine,2023-07-24 11:00:17,‼️ Через війну ціни в Києві виросли на 247%. \...,121536,324,True,154.0,23.0,854.0,74.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
75464,8,Ukraine NOW,2020-03-08 11:10:01,Станом на ранок 8 березня в Україні зареєстро...,56846,105,False,NaN,1.0,NaN,NaN,1.0
75465,7,Ukraine NOW,2020-03-07 15:11:00,​​Міністерство охорони здоров’я України закли...,67616,214,True,NaN,5.0,NaN,NaN,1.0
75466,6,Ukraine NOW,2020-03-07 12:01:00,Міністерство охорони здоров’я України запусти...,62571,145,False,NaN,6.0,NaN,NaN,1.0
75467,5,Ukraine NOW,2020-03-07 11:12:13,Станом на ранок 7 березня 2020 року в Україні...,52026,113,False,1.0,3.0,NaN,NaN,2.0


In [3]:
text_pool = data["text"].tolist()
large_text = " ".join(text_pool).lower()
small_text = large_text[:10000]

with open("large_text.txt", "w", encoding="utf-8") as f:
	f.write(large_text)
	
with open("small_text.txt", "w", encoding="utf-8") as f:
	f.write(small_text)
	
print("Text file created")
print(f"Pool size: {len(text_pool)}")
print(f"Large text size: {len(large_text)}")
print(f"Small text size: {len(small_text)}")

Text file created
Pool size: 75469
Large text size: 25913382
Small text size: 10000


In [12]:
dt = lambda x=time(): time() - x

vocab_size = 2**14 # 16384

spm.SentencePieceTrainer.train(
	input="large_text.txt",
	model_prefix=f"spm_{vocab_size}",
	model_type=["bpe"], # ["bpe", "char", "word", "unigram"]
	vocab_size=vocab_size,
)

print(f"Training took {dt()} seconds")

Training took 13.825632095336914 seconds


In [11]:
sp = spm.SentencePieceProcessor(model_file=f"spm_{vocab_size}.model")

print(f"Vocab size: {sp.get_piece_size()}")

Vocab size: 4096


In [6]:
dt = lambda x=time(): time() - x

# encoded = sp.encode_as_pieces(small_text, enable_sampling=True, alpha=0.1, nbest_size=-1)
encoded = sp.encode(large_text)
print(f"Encoding took {dt()} seconds")
print(f"Encoded size: {len(encoded)}")

Encoding took 30.406882762908936 seconds
Encoded size: 6228905


In [7]:
dt = lambda x=time(): time() - x

decoded = sp.decode(encoded)
print(f"Decoding took {dt()} seconds")
print(f"Decoded size: {len(decoded)}")

Decoding took 5.644878149032593 seconds
Decoded size: 25655470


In [8]:
decoded[:1000]

' ⁇  кадрові зміни у кабміні. новим керівником мінкульту може стати антон дробович, очільник інституту національної пам’яті або олеся островська-люта, директорка «мистецького арсеналу». «дробович — це буде міністр-воїн, який вміє формувати вірні наративи та має зважено доносити історичний багаж. проте до кандидатури островської-лютої більш схильні на банковій», — кажуть наші джерела у парламенті. як нам стало відомо, крісло хитається ще й під міністром молоді і спорту вадимом гутцайтом. times of ukraine  ⁇  не минуло і місяця після звільнення з зсу, як юрій луценко виїхав за кордон. «лікар наполягав на обстеженні у мюнхені», — пояснив ексміністр мвс. 5 липня він полишив службу в армії за станом здоров’я. адже декілька років бореться з онкологією. близький друг петра порошенка оформив інвалідність та натякнув, що повертається у політику. на «гражданці» ексгенпрокурор вже встиг посваритися з «зеленою» депутаткою, дати декілька інтервʼю та звинуватити президента зеленського у початку повн

In [9]:
test_text = "Такий завод зможе випускати 400 танків на рік, задовільнивши потреби ЗСУ. І буде створений, щоб підтримати українців у боротьбі за свободу та демократію, — гендиректор Rheinmetall Армін Паппергер."

encoded = sp.encode(test_text)
decoded = sp.decode(encoded)
print(f"Test text: {test_text}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

Test text: Такий завод зможе випускати 400 танків на рік, задовільнивши потреби ЗСУ. І буде створений, щоб підтримати українців у боротьбі за свободу та демократію, — гендиректор Rheinmetall Армін Паппергер.
Encoded: [8049, 0, 8051, 168, 5870, 4081, 2295, 1111, 21, 4337, 1451, 16, 2039, 8073, 26, 370, 149, 1557, 125, 4013, 8049, 0, 8071, 8049, 0, 467, 5325, 145, 8073, 497, 4998, 899, 17, 7175, 26, 3789, 40, 4272, 172, 8079, 8073, 190, 7705, 2059, 8049, 0, 395, 182, 1813, 8092, 4294, 8049, 0, 8055, 1078, 8049, 0, 8051, 8064, 605, 6840, 8071]
Decoded:  ⁇ акий завод зможе випускати 400 танків на рік, задовільнивши потреби  ⁇ .  ⁇  буде створений, щоб підтримати українців у боротьбі за свободу та демократію, — гендиректор  ⁇ heinmetall  ⁇ рмін  ⁇ аппергер.
